<a href="https://colab.research.google.com/github/ajaysaikiran2208/OpenCV/blob/main/YoloV3_mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Connect google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

# 1) Clone, configure & compile Darknet

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15218, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 15218 (delta 16), reused 27 (delta 16), pack-reused 15185
Receiving objects: 100% (15218/15218), 13.65 MiB | 16.62 MiB/s, done.
Resolving deltas: 100% (10326/10326), done.


In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

2) Configure yolov3.cfg file

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_training.cfg

3) Create .names and .data files

In [ ]:
!echo -e 'Wearing Mask\n Not Wearing Mask' > data/obj.names

In [ ]:
!echo -e 'classes= 2\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

4) Save yolov3_training.cfg and obj.names files in Google drive

In [ ]:
!cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt

5) Create a folder and unzip image dataset

In [ ]:
!mkdir data/obj

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/5f6e33e2d91a1.image.jpg  
  inflating: data/obj/5f6e33e2d91a1.image.txt  
  inflating: data/obj/5f403e14-masksoff-1021x580.jpg  
  inflating: data/obj/5f403e14-masksoff-1021x580.txt  
  inflating: data/obj/8ea05a5e-c1d3-474c-8a4c-f81061afc639.jpg  
  inflating: data/obj/8ea05a5e-c1d3-474c-8a4c-f81061afc639.txt  
  inflating: data/obj/15fb159e-7665-11ea-ab8f-988daf8efd6f_image_hires_155527.jpg  
  inflating: data/obj/15fb159e-7665-11ea-ab8f-988daf8efd6f_image_hires_155527.txt  
  inflating: data/obj/29xp-mask-mediumSquareAt3X.jpg  
  inflating: data/obj/29xp-mask-mediumSquareAt3X.txt  
  inflating: data/obj/97-people-referred-to-Public-Prosecution-for-not-wearing-masks-in-Qatar.jpg  
  inflating: data/obj/97-people-referred-to-Public-Prosecution-for-not-wearing-masks-in-Qatar.txt  
  inflating: data/obj/440nvp.jpg     
  inflating: data/obj/440nvp.txt     
  inflating: data/obj/1679_children-and-face-masks-most-large.txt  
  inf

6) Create train.txt file

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

7) Download pre-trained weights for the convolutional layers file

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-07-10 15:26:25--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  67.0MB/s    in 2.3s    

2021-07-10 15:26:27 (67.0 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



8) Start training

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000009, iou_loss = 0.000000, total_loss = 0.000009 
 total_bbox = 186423, rewritten_bbox = 0.012874 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.674874), count: 3, class_loss = 0.261180, iou_loss = 0.182737, total_loss = 0.443917 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.793618), count: 3, class_loss = 0.280411, iou_loss = 0.111550, total_loss = 0.391962 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.460142), count: 1, class_loss = 0.329929, iou_loss = 0.292523, total_loss = 0.622452 
 total_bbox = 186430, rewritten_bbox = 0.012873 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.761155), count: 7, class_loss = 0.722836, iou_loss = 0.262993, total_loss = 0.985829 
v3 (m